# Mounting Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/new_data.zip

Archive:  /content/drive/MyDrive/new_data.zip
   creating: new_data/
   creating: new_data/test/
   creating: new_data/train/
   creating: new_data/test/image/
   creating: new_data/test/mask/
   creating: new_data/train/image/
   creating: new_data/train/mask/
  inflating: new_data/test/image/01_test_0.png  
  inflating: new_data/test/image/10_test_0.png  
  inflating: new_data/test/image/15_test_0.png  
  inflating: new_data/test/image/09_test_0.png  
  inflating: new_data/test/image/18_test_0.png  
  inflating: new_data/test/image/04_test_0.png  
  inflating: new_data/test/image/03_test_0.png  
  inflating: new_data/test/image/12_test_0.png  
  inflating: new_data/test/image/20_test_0.png  
  inflating: new_data/test/image/17_test_0.png  
  inflating: new_data/test/image/06_test_0.png  
  inflating: new_data/test/image/14_test_0.png  
  inflating: new_data/test/image/08_test_0.png  
  inflating: new_data/test/image/19_test_0.png  
  inflating: new_data/test/image/05_test_0.png  
  i

# Import Libraries

In [ ]:
import os
import skimage.io as io
import skimage.transform as trans
import shutil
import cv2
import matplotlib.pyplot as plt
import pickle
import time
from __future__ import print_function
import glob
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
from tensorflow.keras.models import load_model as load_initial_model
from google.colab.patches import cv2_imshow
import gc

# Handcrafted Metrics For Additional Evaluation

In [ ]:
def dice_coef(y_true, y_pred):
  smooth = 0.0
  y_true_f = keras.flatten(y_true)
  y_pred_f = keras.flatten(y_pred)
  intersection = keras.sum(y_true_f * y_pred_f)
  return (2. * intersection + smooth) / (keras.sum(y_true_f) + keras.sum(y_pred_f) + smooth)

def jacard(y_true, y_pred):

  y_true_f = keras.flatten(y_true)
  y_pred_f = keras.flatten(y_pred)
  intersection = keras.sum ( y_true_f * y_pred_f)
  union = keras.sum ( y_true_f + y_pred_f - y_true_f * y_pred_f)

  return intersection/union

# The Model

In [ ]:
def unet(pretrained_weights = None,input_size = (608,576,1)):
  inputs = Input(input_size)
  conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
  conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
  conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
  conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
  pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
  conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
  conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
  pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
  conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
  conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
  drop4 = Dropout(0.5)(conv4)
  pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

  conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
  conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
  drop5 = Dropout(0.5)(conv5)

  up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
  merge6 = concatenate([drop4,up6], axis = 3)
  conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
  conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

  up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
  merge7 = concatenate([conv3,up7], axis = 3)
  conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
  conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

  up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
  merge8 = concatenate([conv2,up8], axis = 3)
  conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
  conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

  up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
  merge9 = concatenate([conv1,up9], axis = 3)
  conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
  conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
  conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
  conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

  model = Model(inputs,conv10)

  model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy',dice_coef,jacard, tf.keras.metrics.AUC(), tf.keras.metrics.MeanIoU(num_classes=2),
                                                                                      tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    
  #model.summary()

  if(pretrained_weights):
    model.load_weights(pretrained_weights)

  return model

# Generators

In [ ]:
def adjustData(img,mask,flag_multi_class,num_class):
  if(flag_multi_class):
    img = img / 255
    mask = mask[:,:,:,0] if(len(mask.shape) == 4) else mask[:,:,0]
    new_mask = np.zeros(mask.shape + (num_class,))
    for i in range(num_class):
        #for one pixel in the image, find the class in mask and convert it into one-hot vector
        #index = np.where(mask == i)
        #index_mask = (index[0],index[1],index[2],np.zeros(len(index[0]),dtype = np.int64) + i) if (len(mask.shape) == 4) else (index[0],index[1],np.zeros(len(index[0]),dtype = np.int64) + i)
        #new_mask[index_mask] = 1
        new_mask[mask == i,i] = 1
    new_mask = np.reshape(new_mask,(new_mask.shape[0],new_mask.shape[1]*new_mask.shape[2],new_mask.shape[3])) if flag_multi_class else np.reshape(new_mask,(new_mask.shape[0]*new_mask.shape[1],new_mask.shape[2]))
    mask = new_mask
  elif (np.max(img) > 1):
    img = img / 255
    mask = mask /255
    mask[mask > 0.5] = 1
    mask[mask <= 0.5] = 0
  return (img,mask)

In [ ]:
def trainGenerator(batch_size,train_path,image_folder,mask_folder,aug_dict,image_color_mode = "grayscale",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = None,target_size = (608,576),seed = 1):
  image_datagen = ImageDataGenerator(**aug_dict)
  mask_datagen = ImageDataGenerator(**aug_dict)
  image_generator = image_datagen.flow_from_directory(
      train_path,
      classes = [image_folder],
      class_mode = None,
      color_mode = image_color_mode,
      target_size = target_size,
      batch_size = batch_size,
      save_to_dir = save_to_dir,
      save_prefix  = image_save_prefix,
      seed = seed)
  mask_generator = mask_datagen.flow_from_directory(
      train_path,
      classes = [mask_folder],
      class_mode = None,
      color_mode = mask_color_mode,
      target_size = target_size,
      batch_size = batch_size,
      save_to_dir = save_to_dir,
      save_prefix  = mask_save_prefix,
      seed = seed)
  train_generator = zip(image_generator, mask_generator)
  for (img,mask) in train_generator:
    img,mask = adjustData(img,mask,flag_multi_class,num_class)
    yield (img,mask)

In [ ]:
def testGenerator(test_path, target_size = (608,576),flag_multi_class = False,as_gray = True):
  image_datagen = ImageDataGenerator(rescale=1./255)
  mask_datagen = ImageDataGenerator(rescale=1./255)

  for img_name in sorted(os.listdir(test_path + "/images")):
      img = io.imread(os.path.join(test_path + "/images",img_name),as_gray = as_gray)
      img = img / 255
      img = trans.resize(img,target_size)
      img = np.reshape(img,img.shape+(1,)) if (not flag_multi_class) else img
      img = np.reshape(img,(1,)+img.shape)
      yield img

In [ ]:
def testGenerator2(batch_size,test_path,image_folder,mask_folder,aug_dict,image_color_mode = "grayscale",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = None,target_size = (608,576),seed = 1):
  image_datagen = ImageDataGenerator(**aug_dict)
  mask_datagen = ImageDataGenerator(**aug_dict)
  image_generator = image_datagen.flow_from_directory(
      test_path,
      classes = [image_folder],
      class_mode = None,
      color_mode = image_color_mode,
      target_size = target_size,
      batch_size = batch_size,
      save_to_dir = save_to_dir,
      save_prefix  = image_save_prefix,
      seed = seed)
  mask_generator = mask_datagen.flow_from_directory(
      test_path,
      classes = [mask_folder],
      class_mode = None,
      color_mode = mask_color_mode,
      target_size = target_size,
      batch_size = batch_size,
      save_to_dir = save_to_dir,
      save_prefix  = mask_save_prefix,
      seed = seed)
  test_generator = zip(image_generator, mask_generator)
  for (img,mask) in test_generator:
    img,mask = adjustData(img,mask,flag_multi_class,num_class)
    yield (img,mask)

# Utils

In [ ]:
def pad(input_folder, output_folder):
  for file in sorted(os.listdir(input_folder)):
    if '.png' in file:
      tmp = cv2.imread(input_folder + '/' + file, 0)
      tmp = cv2.copyMakeBorder(tmp.copy(),12,12,5,6,cv2.BORDER_CONSTANT,value=(0,0,0))
      io.imsave(output_folder + '/' + file, tmp)
  print('Padding is done.')

In [ ]:
def crop(input_folder, output_folder):
  for file in sorted(os.listdir(input_folder)):
    if '.png' in file:
      tmp = cv2.imread(input_folder + '/' + file, 0)
      tmp = tmp[12:-12, 5:-6]
      io.imsave(output_folder + '/' + file, tmp)
  print('Cropping is done.')

In [ ]:
def labelVisualize(num_class,color_dict,img):
    img = img[:,:,0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i,:] = color_dict[i]
    return img_out 

In [ ]:
def saveResult_drive(save_path,npyfile,flag_multi_class = False,num_class = 2):
  for i,item in enumerate(npyfile):
      img = item[:,:,0]
      img[img >= 0.5] = 255
      img[img < 0.5] = 0
      io.imsave(os.path.join(save_path,"%d.png"%(i)),img)

In [ ]:
def threshold(folder):
  for img in sorted(os.listdir(folder)):
    tmp = cv2.imread(folder + '/' + img, 0)
    _, tmp = cv2.threshold(tmp,115,255,cv2.THRESH_BINARY)
    io.imsave(folder + '/' + img, tmp)

In [ ]:
def set_order(input_folder, output_folder):

  for img in sorted(os.listdir(input_folder)):

    if int(img.split('.')[0]) == 0:
      tmp = cv2.imread(input_folder  + '/' + img, 0)
      io.imsave(output_folder + '/' + str(int(img.split('.')[0])+1) + '.png', tmp)
    elif int(img.split('.')[0]) >= 1 and int(img.split('.')[0]) <= 10:
      tmp = cv2.imread(input_folder  + '/' + img, 0)
      io.imsave(output_folder + '/' + str(int(img.split('.')[0])+9) + '.png', tmp)
    elif int(img.split('.')[0]) >= 13 and int(img.split('.')[0]) <= 19:
      tmp = cv2.imread(input_folder  + '/' + img, 0)
      io.imsave(output_folder + '/' + str(int(img.split('.')[0])-10) + '.png', tmp)
    elif int(img.split('.')[0]) == 11:
      tmp = cv2.imread(input_folder  + '/' + img, 0)
      io.imsave(output_folder + '/' + str(int(img.split('.')[0])-9) + '.png', tmp)
    elif int(img.split('.')[0]) == 12:
      tmp = cv2.imread(input_folder  + '/' + img, 0)
      io.imsave(output_folder + '/' + str(int(img.split('.')[0])+8) + '.png', tmp)

In [ ]:
def dice(true_mask, pred_mask, non_seg_score=1.0):
    """
        Computes the Dice coefficient.
        Args:
            true_mask : Array of arbitrary shape.
            pred_mask : Array with the same shape than true_mask.  
        
        Returns:
            A scalar representing the Dice coefficient between the two segmentations. 
        
    """
    assert true_mask.shape == pred_mask.shape

    true_mask = np.asarray(true_mask).astype(np.bool)
    pred_mask = np.asarray(pred_mask).astype(np.bool)

    # If both segmentations are all zero, the dice will be 1. (Developer decision)
    im_sum = true_mask.sum() + pred_mask.sum()
    if im_sum == 0:
        return non_seg_score

    # Compute Dice coefficient
    intersection = np.logical_and(true_mask, pred_mask)
    return 2. * intersection.sum() / im_sum

In [ ]:
def mean_dice(true_path, pred_path):
  
  sum = 0
  
  for img in sorted(os.listdir(pred_path)):
    
    true_tmp = cv2.imread(true_path + '/' + img, 0)
    pred_tmp = cv2.imread(pred_path + '/' + img, 0)
    
    a = dice(true_tmp, pred_tmp)
    print(a)
    sum += a
  
  return sum/len(os.listdir(true_path))

# DRIVE

In [ ]:
LOG_PATH      = '/content/drive/MyDrive/nerve_segmentation/U-Net/logs'
RESULT_PATH   = '/content/drive/MyDrive/nerve_segmentation/U-Net/test_results'
MODEL_PATH    = '/content/drive/MyDrive/nerve_segmentation/U-Net/models'

TRAIN_PATH    = '/content/new_data/train'
VAL_PATH      = '/content/new_data/test'
TEST_PATH     = '/content/new_data/test'

TMP_TRAIN     = '/content/nerve_segmentation/tmp_train'
TMP_TEST      = '/content/nerve_segmentation/tmp_test'
TMP_VAL       = '/content/nerve_segmentation/tmp_val'
TMP_RESULT    = '/content/nerve_segmentation/tmp_result'

if (os.path.isdir(LOG_PATH) and os.path.isdir(RESULT_PATH) and \
    os.path.isdir(MODEL_PATH) and os.path.isdir(TRAIN_PATH)) and \
    os.path.isdir(TEST_PATH) and os.path.isdir(VAL_PATH) and \
    os.path.isdir(TMP_TRAIN) and os.path.isdir(TMP_TEST) and \
    os.path.isdir(TMP_VAL) and os.path.isdir(TMP_RESULT) == 0:
    raise OSError()

## Train Many Epochs at Once 

In [ ]:
def train_eval_drive(save_name, num_train, num_test, initial_model_path, train_batch = 3, test_batch = 3, epoch = 5):
  
  shutil.rmtree(TMP_TRAIN, ignore_errors=False, onerror=None)
  os.mkdir(TMP_TRAIN)
  os.mkdir(TMP_TRAIN + "/images")
  os.mkdir(TMP_TRAIN + "/labels")

  shutil.rmtree(TMP_TEST, ignore_errors=False, onerror=None)
  os.mkdir(TMP_TEST)
  os.mkdir(TMP_TEST + "/images")
  os.mkdir(TMP_TEST + "/labels")
  
  
  shutil.rmtree(TMP_VAL, ignore_errors=False, onerror=None)
  os.mkdir(TMP_VAL)
  os.mkdir(TMP_VAL + "/images")
  os.mkdir(TMP_VAL + "/labels")
  

  pad(TRAIN_PATH + '/images', TMP_TRAIN + '/images')
  pad(TRAIN_PATH + '/labels', TMP_TRAIN + '/labels')

  pad(TEST_PATH + '/images', TMP_TEST + '/images')
  pad(TEST_PATH + '/labels', TMP_TEST + '/labels')
  
  pad(VAL_PATH + '/images', TMP_VAL + '/images')
  pad(VAL_PATH + '/labels', TMP_VAL + '/labels')
  
  data_gen_args = dict()
  train_generator = trainGenerator(train_batch, TMP_TRAIN, 'images', 'labels', data_gen_args, save_to_dir = None, target_size=(608,576))
  test_generator = testGenerator2(test_batch, TMP_TEST, 'images', 'labels', data_gen_args, save_to_dir = None, target_size=(608,576))

  model = unet(input_size=(608,576,1))
  if initial_model_path != None:
    model.load_weights(initial_model_path)

  model_checkpoint = ModelCheckpoint(MODEL_PATH + "/" + save_name +".hdf5", monitor='loss',verbose=1, save_best_only=True)

  model_history = model.fit_generator(train_generator, steps_per_epoch=num_train//train_batch, epochs=epoch, callbacks=[model_checkpoint],
                                      validation_data=test_generator, validation_steps=num_test//test_batch)
  
  log_file = open(LOG_PATH + "/log_{}.pkl".format(save_name), "wb")#history file
  pickle.dump(model_history.history, log_file)
  log_file.close()

  test_generator_2 = testGenerator(TMP_TEST, target_size=(608,576))
  results = model.predict_generator(test_generator_2,verbose=1)

  shutil.rmtree(TMP_RESULT, ignore_errors=False, onerror=None)
  os.mkdir(TMP_RESULT)
  saveResult_drive(TMP_RESULT, results)
  
  os.mkdir(RESULT_PATH + '/' + save_name)
  crop(TMP_RESULT, RESULT_PATH + '/' + save_name)

  threshold(RESULT_PATH + '/' + save_name)

  shutil.rmtree(RESULT_PATH + '/download', ignore_errors=False, onerror=None)
  os.mkdir(RESULT_PATH + '/download')
  set_order(RESULT_PATH + '/' + save_name, RESULT_PATH + '/download')
  return results

In [ ]:
train_sample_number = len(os.listdir(TRAIN_PATH + '/images'))
test_sample_number  = len(os.listdir(TEST_PATH + '/images'))

In [ ]:
test_sample_number

20

In [ ]:
SAVE_NAME = 'my_model2'
INITIAL_MODEL_PATH = None
EPOCH = 30

In [ ]:
res = train_eval_drive(SAVE_NAME, initial_model_path= INITIAL_MODEL_PATH, epoch= EPOCH, train_batch = 3, test_batch = 3, num_train = train_sample_number, num_test= test_sample_number)

Padding is done.
Padding is done.
Padding is done.
Padding is done.
Padding is done.
Padding is done.
Found 80 images belonging to 1 classes.
Found 80 images belonging to 1 classes.
Epoch 1/30


/usr/local/lib/python3.9/dist-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
<ipython-input-70-e5e6484cf365>:39: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_history = model.fit_generator(train_generator, steps_per_epoch=num_train//train_batch, epochs=epoch, callbacks=[model_checkpoint],


26/26 [==============================] - ETA: 0s - loss: 0.6942 - accuracy: 0.8866 - dice_coef: 0.1401 - jacard: 0.0754 - auc_7: 0.5004 - mean_io_u_7: 0.4592 - precision_7: 0.0765 - recall_7: 0.0353Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.

Epoch 1: loss improved from inf to 0.69422, saving model to /content/drive/MyDrive/nerve_segmentation/U-Net/models/my_model2.hdf5
26/26 [==============================] - 45s 2s/step - loss: 0.6942 - accuracy: 0.8866 - dice_coef: 0.1401 - jacard: 0.0754 - auc_7: 0.5004 - mean_io_u_7: 0.4592 - precision_7: 0.0765 - recall_7: 0.0353 - val_loss: 0.6921 - val_accuracy: 0.9188 - val_dice_coef: 0.1397 - val_jacard: 0.0751 - val_auc_7: 0.5000 - val_mean_io_u_7: 0.4594 - val_precision_7: 0.0000e+00 - val_recall_7: 0.0000e+00
Epoch 2/30
26/26 [==============================] - ETA: 0s - loss: 0.6915 - accuracy: 0.9177 - dice_coef: 0.1413 - jacard: 0.0761 - auc_7: 0.5000 - mean_io_u_7: 0.4588 - precision_7: 0.0000e+00 - r

<ipython-input-70-e5e6484cf365>:47: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  results = model.predict_generator(test_generator_2,verbose=1)


20/20 [==============================] - 3s 143ms/step


<ipython-input-59-240b4496da9c>:6: UserWarning: /content/nerve_segmentation/tmp_result/0.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d.png"%(i)),img)
<ipython-input-59-240b4496da9c>:6: UserWarning: /content/nerve_segmentation/tmp_result/1.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d.png"%(i)),img)
<ipython-input-59-240b4496da9c>:6: UserWarning: /content/nerve_segmentation/tmp_result/2.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d.png"%(i)),img)
<ipython-input-59-240b4496da9c>:6: UserWarning: /content/nerve_segmentation/tmp_result/3.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d.png"%(i)),img)
<ipython-input-59-240b4496da9c>:6: UserWarning: /content/nerve_segmentation/tmp_result/4.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d.png"%(i)),img)
<ipython-input-59-240b4496da9c>:6: UserWarning: /content/nerve_segmentation/tmp_result/5.png is a low contrast image
  io.imsave(os.path.join(sav

Cropping is done.


<ipython-input-13-a26a2ee73cd2>:5: UserWarning: /content/drive/MyDrive/nerve_segmentation/U-Net/test_results/my_model2/2.png is a low contrast image
  io.imsave(folder + '/' + img, tmp)
<ipython-input-13-a26a2ee73cd2>:5: UserWarning: /content/drive/MyDrive/nerve_segmentation/U-Net/test_results/my_model2/3.png is a low contrast image
  io.imsave(folder + '/' + img, tmp)
<ipython-input-13-a26a2ee73cd2>:5: UserWarning: /content/drive/MyDrive/nerve_segmentation/U-Net/test_results/my_model2/4.png is a low contrast image
  io.imsave(folder + '/' + img, tmp)
<ipython-input-13-a26a2ee73cd2>:5: UserWarning: /content/drive/MyDrive/nerve_segmentation/U-Net/test_results/my_model2/5.png is a low contrast image
  io.imsave(folder + '/' + img, tmp)
<ipython-input-13-a26a2ee73cd2>:5: UserWarning: /content/drive/MyDrive/nerve_segmentation/U-Net/test_results/my_model2/6.png is a low contrast image
  io.imsave(folder + '/' + img, tmp)
<ipython-input-13-a26a2ee73cd2>:5: UserWarning: /content/drive/MyDrive

In [ ]:
e = res[0]*255
e

array([[[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       ...,

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]], dtype=float32)

In [ ]:
log_file = open(LOG_PATH + "/log_" + SAVE_NAME + ".pkl" , "rb")
output = pickle.load(log_file)
i = 0
for key, value in output.items():
  print(key + " --> " + str(value[EPOCH-1]))
  i = i+1
print(50*"-")

loss --> 0.25439178943634033
accuracy --> 0.9175898432731628
dice_coef --> 0.1351488083600998
jacard --> 0.07252132892608643
auc_1 --> 0.7009954452514648
mean_io_u_1 --> 0.4587949216365814
precision_1 --> 0.0
recall_1 --> 0.0
val_loss --> 0.27388301491737366
val_accuracy --> 0.9194892644882202
val_dice_coef --> 0.1702936291694641
val_jacard --> 0.0930793285369873
val_auc_1 --> 0.7165288329124451
val_mean_io_u_1 --> 0.4597446322441101
val_precision_1 --> 0.0
val_recall_1 --> 0.0
--------------------------------------------------


In [ ]:
mean_dice_coef = mean_dice(TEST_PATH +   '/labels', 
                           RESULT_PATH + '/download')

In [ ]:
mean_dice_coef

## Train with Loop

In [ ]:
def train_eval_drive(save_name, initial_model_name, n_samples, train_batch = 3, test_batch = 3, epochs = 5):
  
  shutil.rmtree(TMP_TRAIN, ignore_errors=False, onerror=None)
  os.mkdir(TMP_TRAIN)
  os.mkdir(TMP_TRAIN + "/images")
  os.mkdir(TMP_TRAIN + "/labels")

  shutil.rmtree(TMP_TEST, ignore_errors=False, onerror=None)
  os.mkdir(TMP_TEST)
  os.mkdir(TMP_TEST + "/images")
  os.mkdir(TMP_TEST + "/labels")
  
  pad(TRAIN_PATH + '/images', TMP_TRAIN + '/images')
  pad(TRAIN_PATH + '/labels', TMP_TRAIN + '/labels')

  pad(TEST_PATH + '/images', TMP_TEST + '/images')
  pad(TEST_PATH + '/labels', TMP_TEST + '/labels')
  
  data_gen_args = dict()
  train_generator = trainGenerator(train_batch, TMP_TRAIN, 'images', 'labels', data_gen_args, save_to_dir = None, target_size=(608,576))
  test_generator = testGenerator2(test_batch, TMP_TEST, 'images', 'labels', data_gen_args, save_to_dir = None, target_size=(608,576))

  for epoch in range(epochs):
  
    model = unet(input_size=(608,576,1))
    if epoch != 0:
      model.load_weights(f'{MODEL_PATH}/{save_name +"_"+ str(epoch-1)}.hdf5')
    else:
      if initial_model_name != None:
        model.load_weights(f'{MODEL_PATH}/{initial_model_name}.hdf5')

    model_checkpoint = ModelCheckpoint(MODEL_PATH + "/" + save_name + "_" + str(epoch) +".hdf5", monitor='loss',verbose=1, save_best_only=True)

    model_history = model.fit_generator(train_generator, steps_per_epoch=n_samples//train_batch, epochs=1, callbacks=[model_checkpoint],
                                      validation_data=test_generator, validation_steps=20//test_batch)
  
    log_file = open(LOG_PATH + "/log_{}.pkl".format(save_name + str(epoch)), "wb")#history file
    pickle.dump(model_history.history, log_file)
    log_file.close()

    test_generator_2 = testGenerator(TMP_TEST, target_size=(608,576))
    results = model.predict_generator(test_generator_2,verbose=1)

    shutil.rmtree(TMP_RESULT, ignore_errors=False, onerror=None)
    os.mkdir(TMP_RESULT)
    saveResult_drive(TMP_RESULT, results)
    
    os.mkdir(RESULT_PATH + '/' + save_name + str(epoch))
    crop(TMP_RESULT, RESULT_PATH + '/' + save_name + str(epoch))

    threshold(RESULT_PATH + '/' + save_name + str(epoch))

    os.mkdir(RESULT_PATH + '/download' + "_"+save_name + str(epoch))
    set_order(RESULT_PATH + '/' + save_name + str(epoch), RESULT_PATH + '/download' + "_"+save_name + str(epoch))

    mean_dice_coef = mean_dice(TEST_PATH   + '/labels', 
                               RESULT_PATH + '/download_' + save_name + str(epoch))
    print(f"\nMean dice coeff at epoch {epoch}: {mean_dice_coef}\n\n")

In [ ]:
train_sample_number = len(os.listdir(TRAIN_PATH + '/images'))

In [ ]:
SAVE_NAME = 'my_model'
initial_model_name = None

train_eval_drive(save_name=SAVE_NAME, initial_model_name = initial_model_name, n_samples= train_sample_number, epochs=30)

In [ ]:
!git clone https://github.com/nikhilroxtomar/Retina-Blood-Vessel-Segmentation-in-PyTorch.git

Cloning into 'Retina-Blood-Vessel-Segmentation-in-PyTorch'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 105 (delta 17), reused 98 (delta 16), pack-reused 1
Receiving objects: 100% (105/105), 8.87 MiB | 6.64 MiB/s, done.
Resolving deltas: 100% (17/17), done.


In [ ]:
%cd /content/Retina-Blood-Vessel-Segmentation-in-PyTorch/UNET

/content/Retina-Blood-Vessel-Segmentation-in-PyTorch/UNET


In [ ]:
!ls

data.py  img	  model.py  __pycache__  results  train.py
files	 loss.py  new_data  README.md	 test.py  utils.py


In [ ]:
!python3 test.py

100% 20/20 [00:10<00:00,  1.97it/s]
Jaccard: 0.6634 - F1: 0.7974 - Recall: 0.7771 - Precision: 0.8240 - Acc: 0.9657
FPS:  12.45600955188492


In [ ]:
!python3 test.py

100% 20/20 [00:10<00:00,  1.95it/s]
Jaccard: 0.6634 - F1: 0.7974 - Recall: 0.7769 - Precision: 0.8245 - Acc: 0.9657
FPS:  12.646520838185864


In [ ]:
!python3 train.py

Dataset Size:
Train: 80 - Valid: 20

Valid loss improved from inf to 1.3519. Saving checkpoint: /content/drive/MyDrive/nerve_segmentation/checkpoint_new.pth
Epoch: 01 | Epoch Time: 0m 34s
	Train Loss: 1.168
	 Val. Loss: 1.352

Valid loss improved from 1.3519 to 0.9793. Saving checkpoint: /content/drive/MyDrive/nerve_segmentation/checkpoint_new.pth
Epoch: 02 | Epoch Time: 0m 33s
	Train Loss: 0.983
	 Val. Loss: 0.979

Valid loss improved from 0.9793 to 0.8961. Saving checkpoint: /content/drive/MyDrive/nerve_segmentation/checkpoint_new.pth
Epoch: 03 | Epoch Time: 0m 34s
	Train Loss: 0.923
	 Val. Loss: 0.896

Valid loss improved from 0.8961 to 0.8537. Saving checkpoint: /content/drive/MyDrive/nerve_segmentation/checkpoint_new.pth
Epoch: 04 | Epoch Time: 0m 34s
	Train Loss: 0.877
	 Val. Loss: 0.854

Valid loss improved from 0.8537 to 0.8122. Saving checkpoint: /content/drive/MyDrive/nerve_segmentation/checkpoint_new.pth
Epoch: 05 | Epoch Time: 0m 34s
	Train Loss: 0.835
	 Val. Loss: 0.812

V